In [1]:
!git clone https://github.com/inttx/DLAM_SealedSurfaces.git
%cd DLAM_SealedSurfaces

Cloning into 'DLAM_SealedSurfaces'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 124 (delta 71), reused 77 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (124/124), 163.94 KiB | 6.83 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/home/janina/PycharmProjects/DLAM_SealedSurfaces/DLAM_SealedSurfaces


In [5]:
!pip install -r requirements.txt
%cd src

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
    --------------------------------------- 0.5/38.9 MB 4.2 MB/s eta 0:00:10
   - -------------------------------------- 1.3/38.9 MB 3.7 MB/s eta 0:00:11
   -- ------------------------------------- 2.1/38.9 MB 3.4 MB/s eta 0:00:11
   -- ------------------------------------- 2.6/38.9 MB 3.4 MB/s eta 0:00:11
   --- ------------------------------------ 3.4/38.9 MB 3.4 MB/s eta 0:00:11
   ---- ----------------------------------- 4.2/38.9 MB 3.4 MB/s eta 0:00:11
   ---- ----------------------------------- 4.7/38.9 MB 3.4 MB/s eta 0:00:11
   ----- ---------------------------------- 5.5/38.9 MB 3.4 M

  error: subprocess-exited-with-error
  
  × Building wheel for stringzilla (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [10 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\cli
      copying cli\split.py -> build\lib.win-amd64-cpython-312\cli
      copying cli\wc.py -> build\lib.win-amd64-cpython-312\cli
      copying cli\__init__.py -> build\lib.win-amd64-cpython-312\cli
      running build_ext
      building 'stringzilla' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for stringzilla

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Fa

In [8]:
from settings import *
from dataset import PotsdamDataset, get_data_loaders
from models import get_trained_custom_resnet18, get_trained_deeplabv3_resnet101, get_trained_segformer_model
from evaluate import evaluate_segmentation

ModuleNotFoundError: No module named 'albumentations'

In [4]:
try:
    import os
    import shutil
    from google.colab import drive
    MOUNTPOINT = '/content/drive/'
    drive.mount(MOUNTPOINT)

    IMAGE_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '2_Ortho_RGB') # TODO adjust to your path
    LABEL_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '5_Labels_all') # TODO adjust to your path

    EVAL_PATH = MOUNTPOINT + 'MyDrive/DLAM/eval/'
except:
    IMAGE_PATH = '../data/2_Ortho_RGB' # TODO adjust to your path
    LABEL_PATH = '../data/5_Labels_all' # TODO adjust to your path
    EVAL_PATH = '../eval/'
os.makedirs(EVAL_PATH, exist_ok=True)

In [5]:
# Hyperparameters
patch_size = 250
stride = 250
batch_size = 8
classification_mode = "single" # or "combined"
num_classes = PotsdamDataset.get_num_classes(classification_mode)
class_names = PotsdamDataset.CLASS_NAMES_SINGLE if classification_mode == "single" else PotsdamDataset.CLASS_NAMES_COMBINED

# Train resnet18

In [5]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=[0.8, 0.1, 0.1],
                     batch_size=batch_size,
                     device=DEVICE,
                     mode=classification_mode,
                     pin_memory=True,
                     num_workers=2,
                     seed=42)

model = get_trained_custom_resnet18(model_path='../models/resnet18.pth', patch_size=patch_size, num_classes=num_classes, device=DEVICE)
evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='ResNet18', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'resnet18.svg')

Evaluating: 100%|██████████| 274/274 [01:25<00:00,  3.20it/s]
/home/janina/PycharmProjects/DLAM_SealedSurfaces/src/evaluate.py:83: RuntimeWarning: invalid value encountered in divide
  per_class_iou = np.where(union > 0, intersection / union, 0.0)


Evaluation completed.
Pixel Accuracy: 0.6414
Mean IoU: 0.3597
Per-class IoU:
  Class 0: 0.5105
  Class 1: 0.5755
  Class 2: 0.4061
  Class 3: 0.3663
  Class 4: 0.2998
  Class 5: 0.0000


# Train DeepLabV3 resnet101 baseline

In [5]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=[0.8, 0.1, 0.1],
                     batch_size=batch_size,
                     device=DEVICE,
                     mode=classification_mode,
                     pin_memory=True,
                     num_workers=2,
                     seed=42)

model = get_trained_deeplabv3_resnet101(model_path='../models/deeplabv3_resnet101.pth', num_classes=num_classes, device=DEVICE)

evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='DeepLabV3', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'deeplabv3_resnet101.svg')

Evaluating:   0%|          | 1/274 [00:02<10:08,  2.23s/it]

Logits shape: torch.Size([8, 6, 250, 250]), Labels shape: torch.Size([8, 250, 250])
Predictions shape: torch.Size([8, 250, 250])


Evaluating:   1%|          | 2/274 [00:04<10:09,  2.24s/it]

Logits shape: torch.Size([8, 6, 250, 250]), Labels shape: torch.Size([8, 250, 250])
Predictions shape: torch.Size([8, 250, 250])


Evaluating:   1%|          | 2/274 [00:05<13:08,  2.90s/it]


KeyboardInterrupt: 

# Train SegFormer

In [6]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=[0.8, 0.1, 0.1],
                     batch_size=batch_size,
                     device=DEVICE,
                     mode=classification_mode,
                     pin_memory=True,
                     num_workers=2,
                     seed=42)

model = get_trained_segformer_model('../models/segformer.pth', num_classes=num_classes, device=DEVICE)

evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='SegFormer', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'segformer.svg')

Building index: 100%|██████████| 24/24 [00:00<00:00, 319566.02it/s]
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([6]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([6, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 274/274 [00:47<00:00,  5.79it/s]
/home/janina/PycharmProjects/DLAM_SealedSurfaces/src/evaluate.py:81: RuntimeWarning: invalid value encountered in divide
  per_class_iou = np.where(union > 0, intersection / union, 0.0)


Evaluation completed.
Pixel Accuracy: 0.8490
Mean IoU: 0.5763
Per-class IoU:
  Class 0: 0.7717
  Class 1: 0.8496
  Class 2: 0.6736
  Class 3: 0.6913
  Class 4: 0.4715
  Class 5: 0.0000
